In [1]:
# Install necessary library
!pip install gradio

# Import libraries
import requests
import gradio as gr
from getpass import getpass


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
# Prompt user to securely input the API key
API_KEY = getpass("Enter your Gemini API key: ")

# Function to call the Gemini API
def call_gemini_api(prompt, max_tokens=300):
    API_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent"
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    url_with_key = f"{API_ENDPOINT}?key={API_KEY}"
    response = requests.post(url_with_key, json=payload, headers=headers)

    # Log the raw response
    print("Response Debug:", response.status_code, response.text, flush=True)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")


Enter your Gemini API key: ··········


In [3]:
# Gradio Interface with Enhanced UI and No Loading Text
with gr.Blocks(css="""
    .gradio-container {
        background: linear-gradient(to bottom right, #fef6e4, #f9f9f9);
        font-family: 'Roboto', sans-serif;
        color: #4a4a4a;
        overflow-y: auto;
        max-height: 100vh;
    }
    .gr-button {
        background-color: #f4a261 !important; /* Orange by default */
        border: none !important;
        border-radius: 8px !important;
        color: #ffffff !important;
        font-weight: bold !important;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
        transition: background-color 0.3s ease-in-out;
        padding: 12px 20px !important; /* Adjust padding for larger button */
    }
    .gr-button:hover {
        background-color: #219ebc !important; /* Blue on hover */
    }
    .gr-textbox textarea {
        font-size: 16px !important;
        padding: 12px !important;
    }
""") as app:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: #219ebc;">🌟 LinkedIn Content Generator</h1>
            <p style="font-size: 16px; color: #4a4a4a;">Generate engaging LinkedIn posts effortlessly!</p>
        </div>
        """
    )

    with gr.Row():
        topic_input = gr.Textbox(
            label="Enter Topic",
            placeholder="e.g., Self-development, Leadership",
            elem_id="topic-input"
        )
        generate_ideas_button = gr.Button("Generate Ideas", elem_classes=["gr-button"])

    with gr.Row():
        ideas_output = gr.Radio(
            label="Select an Idea",
            choices=[],
            interactive=True,
            elem_id="ideas-output"
        )

    post_output = gr.Textbox(
        label="Generated LinkedIn Post",
        lines=10,
        show_copy_button=True,
        elem_id="post-output",
        visible=False  # Initially hidden
    )

    # Function to generate LinkedIn ideas
    def generate_ideas(topic):
        try:
            prompt = (
                f"You are a LinkedIn content expert. Generate 10 unique LinkedIn post ideas about '{topic}'. Avoid hashtags, special formatting, or emojis. Each idea should be a simple, brief (5-8 words), clear, and engaging sentence."
            )
            response = call_gemini_api(prompt)

            # Extract and return ideas based on the response structure
            if "candidates" in response:
                ideas_text = response["candidates"][0]["content"]["parts"][0]["text"]
                ideas = [idea.strip() for idea in ideas_text.split('\n') if idea.strip()]
                return gr.update(choices=ideas, value=None)  # Update Radio with choices
            else:
                print("Unexpected Response Format:", response)
                return gr.update(choices=["Error: Unexpected response format. Check API response."], value=None)
        except Exception as e:
            return gr.update(choices=[f"Error: {str(e)}"], value=None)

    # Function to generate a full LinkedIn post based on selected idea
    def generate_post(idea):
        if not idea:
            return gr.update(value="Please select an idea first.", visible=True)
        try:
            prompt = (
                f"Write an engaging LinkedIn post based on the idea '{idea}'. Include:\n"
                f"1. An attention-grabbing hook\n"
                f"2. 2-3 key points or insights\n"
                f"3. A personal reflection or practical takeaway\n"
                f"4. A call to action\n"
                f"5. 3-4 relevant hashtags\n"
                f"6. In place of any bullet points, if needed replace them with numbers 1, 2 and so on\n"
                f"7. Appropriate emojis\n\n"
                f"Make it sound natural, professional, and engaging, with light formatting, no bolding of any words and emojis for better readability."
            )
            response = call_gemini_api(prompt, max_tokens=300)

            # Extract and return the full post with original formatting
            if "candidates" in response:
                return gr.update(
                    value=response["candidates"][0]["content"]["parts"][0]["text"], visible=True
                )
            else:
                print("Unexpected Response Format:", response)
                return gr.update(value="Error: Unexpected response format. Check API response.", visible=True)
        except Exception as e:
            return gr.update(value=f"Error: {str(e)}", visible=True)

    # Link functionality
    generate_ideas_button.click(
        fn=generate_ideas,
        inputs=topic_input,
        outputs=ideas_output
    )

    ideas_output.change(
        fn=generate_post,
        inputs=ideas_output,
        outputs=post_output
    )

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d731e40dfec0c6c04e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
